# Creating a logistic regression to predict absenteeism

## Import relevant libraries

In [1]:
import pandas as pd
import numpy as np

## Import the data

In [2]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')
data_preprocessed

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,2,179,22,40,237.656,22,1,2,0,8
696,1,0,0,0,5,2,225,26,28,237.656,24,0,1,2,3
697,1,0,0,0,5,3,330,16,28,237.656,25,1,0,0,8
698,0,0,0,1,5,3,235,16,32,237.656,25,1,0,0,2


## create the target

In [3]:
# we will take the median value of the 'Absenteeism time in Hours' and use it as a cut-off line
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [4]:
# assign <=3, moderately absent (or 0), and >=4 excessively absent (0r 1)
targets = np.where(data_preprocessed['Absenteeism Time in Hours']>data_preprocessed['Absenteeism Time in Hours'].median(),1,0)

In [5]:
data_preprocessed['Excessive Absenteeism']=targets
data_preprocessed.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


## comment on the targets

In [6]:
# by using the median(), we implicitly balanced the dataset
# this prevent model from learning to output only 0's and 1's
targets.sum()/targets.shape[0]

# balance of 45:55 is sufficient

0.45571428571428574

## create the checkpoint

In [7]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'],axis=1)

## select input for the regression

In [8]:
data_with_targets.shape

(700, 15)

In [9]:
unscaled_inputs=data_with_targets.iloc[:,0:-1]

## standarise the data

In [48]:
## from sklearn.preprocessing import StandardScaler

## absenteeism_scaler = StandardScaler()

# This section is commented out since we will use custom scaler in order to 
# avoid the standarisation of the dummy variables

In [58]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [59]:
unscaled_inputs.columns.values

array(['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Month Value',
       'Day of the Week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [60]:
## omit the dummy variables
columns_to_scale = ['Month Value',
       'Day of the Week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index',
       'Children', 'Pets']

In [61]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [62]:
## this line calculate and store the mean and the std and stored in absenteeism_scaler object
absenteeism_scaler.fit(unscaled_inputs)

C:\Users\yu298\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CustomScaler(columns=['Month Value', 'Day of the Week',
                      'Transportation Expense', 'Distance to Work', 'Age',
                      'Daily Work Load Average', 'Body Mass Index', 'Children',
                      'Pets'],
             copy=None, with_mean=None, with_std=None)

In [63]:
## apply scaling mechanism
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)
scaled_inputs

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,-0.683704,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-0.683704,-1.574681,-1.141882,2.130803,-0.806331,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.007725,-0.654143,1.426749,0.248310,-0.806331,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.668253,0.854936,-1.682647,0.405184,-0.806331,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,0.668253,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.007725,-0.654143,-0.533522,0.562059,-0.853789,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,-0.007725,0.040034,-0.263140,-1.320435,-0.853789,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,0.668253,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.668253,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,1,-0.919030,-0.589690


In [64]:
scaled_inputs.shape

(700, 14)

## split the data into train& test and shuffle

### import the relevant module

In [65]:
from sklearn.model_selection import train_test_split

### split

In [66]:
## split the inputs into train and test variables
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20)

## Logistic regression with sklearn

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

### training the model

In [68]:
reg = LogisticRegression()

In [69]:
reg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [70]:
## returns the mean accuracy on the given test data and labels
reg.score(x_train,y_train)

0.775

## manually check the accuracy

In [71]:
model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [72]:
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [73]:
## counting the number of entries that model_output == y_train
np.sum((model_outputs==y_train))

434

In [74]:
## total number of the entries
model_outputs.shape[0]

560

In [75]:
## calculate the accuracy
np.sum((model_outputs==y_train))/model_outputs.shape[0]

0.775

## finding intercept and coefficients

In [76]:
## finding intercept
reg.intercept_

array([-1.6561092])

In [77]:
## finding coefficients
reg.coef_

array([[ 2.80096498e+00,  9.34857518e-01,  3.09561645e+00,
         8.56587468e-01,  1.66248119e-01, -8.43703301e-02,
         6.12732578e-01, -7.79685996e-03, -1.65922708e-01,
        -1.47005122e-04,  2.71811477e-01, -2.05738037e-01,
         3.61989880e-01, -2.85510745e-01]])

In [78]:
## we want to know which coefficents refers to which variable
feature_name = unscaled_inputs.columns.values
feature_name

array(['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Month Value',
       'Day of the Week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [79]:
# summarise feature names and their coefficients by creating the new dataframe
summary_table = pd.DataFrame(columns=['Feature name'],data=feature_name)

# add new column of coefficient 
summary_table['coefficient'] = np.transpose(reg.coef_)
summary_table

,Feature name,coefficient
0,Reason 1,2.800965
1,Reason 2,0.934858
2,Reason 3,3.095616
3,Reason 4,0.856587
4,Month Value,0.166248
5,Day of the Week,-0.084370
6,Transportation Expense,0.612733
7,Distance to Work,-0.007797
8,Age,-0.165923
9,Daily Work Load Average,-0.000147


In [80]:
## adding intercept to the table
summary_table.index = summary_table.index + 1
summary_table.loc[0]= ['Intercept',reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,coefficient
0,Intercept,-1.656109
1,Reason 1,2.800965
2,Reason 2,0.934858
3,Reason 3,3.095616
4,Reason 4,0.856587
5,Month Value,0.166248
6,Day of the Week,-0.084370
7,Transportation Expense,0.612733
8,Distance to Work,-0.007797
9,Age,-0.165923


## interpreting the coefficient

In [81]:
## finding odds by taking the exponential
summary_table['odd ratio'] = np.exp(summary_table.coefficient)

In [82]:
# sort the table in a descending order
summary_table.sort_values('odd ratio',ascending=False)

,Feature name,coefficient,odd ratio
3,Reason 3,3.095616,22.100858
1,Reason 1,2.800965,16.460523
2,Reason 2,0.934858,2.546851
4,Reason 4,0.856587,2.355110
7,Transportation Expense,0.612733,1.845467
13,Children,0.361990,1.436184
11,Body Mass Index,0.271811,1.312340
5,Month Value,0.166248,1.180866
10,Daily Work Load Average,-0.000147,0.999853
8,Distance to Work,-0.007797,0.992233


In [83]:
## interpretation here:
# A feature is not particularly important if:
#   its coefficeint is around 0
#   its odd ratio is around 1
# For a unit change in the standarised feature, the odds increase by a 
# multiple equal to the odds ratio (1 = no change)

# daily work load average, distance to work, day of the week seem to not affect the model much

### standarising only the numerical variables

When the variables are standarised, so was all the dummy variables.  
However, this make it not possible to compare between the dummy variables.
e.g. On the table avobe, we could have said Reason 1 is 7.98 times more likely 
than reason 0 (which is the benchmark)
To solve this, we go back to the step where we standarised the variables.

## Backword elimination

The idea is that we can simplify our model by removing all features which have close to no contribution to the model.

Which we have the p-values, we get rid of all coeffieints with p-values > 0.05

Although p-value is not included in sklearn ,if the weight is small enough, it wont make a difference anyway.

If we remove these variables, the rest of our model should not really change in terms of coefficient values.

We now create the copy of this file, in which we will aplly the backward elimination